# Formula 1 data analysis application season 2025
This is the 2025 version

This time, I use fastf1 for race data

<h2 style="color: red;">DON'T FORGET TO UPDATE RACE INDEX</h2>
<h2 style="color: red;">RUN GLOBALDATA.PY FIRST UPDATING THE RACE INDEX THERE TOO</h2>

In [ ]:
from racenameindex import RaceNameIndex

race_idx = RaceNameIndex.Melbourne
race_year = 2025

In [2]:
# BackEnd
import fastf1 as ff1
import fastf1.plotting as ff1plt
import numpy as np
import pandas as pd
import plotly.express as px
from fastf1.core import Laps
import globaldata as gd
import json

# init
ff1plt.setup_mpl(mpl_timedelta_support=True, misc_mpl_mods=False, color_scheme='fastf1')

# last race data
race = ff1.get_session(race_year, race_idx, 'R')
race.load()
driver_laps = race.laps.pick_drivers(race.drivers)
driver_laps = driver_laps.reset_index()[driver_laps["IsAccurate"] == True] 
driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
driver_laps["Time(s)"] = driver_laps["Time"].dt.total_seconds()
average_lap_time = driver_laps.groupby("Driver")["LapTime(s)"].mean().reset_index()
fastest = average_lap_time["LapTime(s)"].min()
average_lap_time['LapTimeDelta'] = average_lap_time['LapTime(s)'] - fastest
finishing_order = [race.get_driver(i)["Abbreviation"] for i in race.drivers]
average_lap_time["Position"] = average_lap_time["Driver"].apply(lambda x: finishing_order.index(x) + 1)
average_lap_time = average_lap_time.sort_values(by="Position")
# average_lap_time["Driver"] = average_lap_time["Driver"].str

# qualifying data
quali_session = ff1.get_session(race_year, race_idx, 'Q')
quali_session.load()
quali_data = quali_session.laps.pick_drivers(quali_session.drivers)
quali_data = quali_data.reset_index()[quali_data["IsAccurate"] == True]
list_fastest_laps = list()
for drv in quali_data["Driver"].unique():
    drvs_fastest_lap = quali_data.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)
fastest_laps = Laps(list_fastest_laps) \
    .sort_values(by='LapTime') \
    .reset_index(drop=True)
pole_lap = fastest_laps.pick_fastest()
fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']
fastest_laps["LapTime"] = fastest_laps["LapTime"].dt.total_seconds()
fastest_laps["LapTimeDelta"] = fastest_laps["LapTimeDelta"].dt.total_seconds()
# fastest_laps["Driver"] = fastest_laps["Driver"].str

# backend for 2 fastest drivers comparison
result = quali_session.results
r1 = result.loc[result['Position'] == 1]
r2 = result.loc[result['Position'] == 2]
lap = quali_session.laps.pick_driver(r1['Abbreviation'][0]).pick_fastest()
lap2 = quali_session.laps.pick_driver(r2['Abbreviation'][0]).pick_fastest()
colormap = {r1['Abbreviation'][0]: ff1plt.get_driver_color(r1['Abbreviation'][0], quali_session), r2['Abbreviation'][0]: ff1plt.get_driver_color(r2['Abbreviation'][0], quali_session)}
tel = lap.get_telemetry()
tel2 = lap2.get_telemetry()
driver = lap.Driver
driver2 = lap2.Driver
x = np.array(tel['X'].values)
y = np.array(tel['Y'].values)
x2 = np.array(tel2['X'].values)
y2 = np.array(tel2['Y'].values)
def cumDist(x, y):
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    cumdist = np.cumsum(dist)
    return np.concatenate(([0], cumdist))
cd = cumDist(x, y)
cd2 = cumDist(x2, y2)
time = np.array(tel['Time'].values)
time2 = np.array(tel2['Time'].values)
dif = []
for i in range(len(cd)):
    cpd = cd[i]
    t = time[i]
    if t not in time2:
        cpd2 = np.interp(np.float64(t), np.float64(time2), np.float64(cd2))
    else:
        cpd2 = cd2[np.where(time2 == t)[0][0]]
    if cpd2 > cpd:
        dif.append(driver2)
    else:
        dif.append(driver)
qualy2 = pd.DataFrame({'X': x, 'Y': y, 'Driver': dif})

# gap to winner
winner_lap = driver_laps[driver_laps["Position"] == 1].iloc[-1]
winner_driver = winner_lap["Driver"]
winner_times = driver_laps[driver_laps["Driver"] == winner_driver][["LapNumber", "Time(s)"]].set_index("LapNumber")
driver_laps = driver_laps.merge(winner_times, on="LapNumber", suffixes=("", "_Winner"))
driver_laps['GapToWinner'] = driver_laps['Time(s)'] - driver_laps['Time(s)_Winner']

# stints
stints = driver_laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stint_idxs = {drv: [1] for drv in stints["Driver"].unique()}
# stints["Driver"] = stints["Driver"].str

# colors
driver_colors = ff1plt.get_driver_color_mapping(session=race)
team_colors = {ff1plt.get_team_name_by_driver(driver, race): ff1plt.get_team_color(ff1plt.get_team_name_by_driver(driver, race), race) for driver in ff1plt.list_driver_abbreviations(race)}
compound_colors = {}
for compound in stints["Compound"].unique():
    compound_colors[compound] = ff1plt.get_compound_color(compound, session=race)

# global year driver
print(race_idx)
global_df = gd.update_data(race_year, race_idx)

# global year team
global_team_df = global_df.groupby(["raceIndex", "team"]).sum().reset_index()
global_team_df = global_team_df.drop(columns=["driver", "quali", "result"])

# leclerc
df_lec = global_df[global_df["driver"] == "LEC"]
dico = {"position": [], "qualy": [], "result": []}
for i in range(1, 21):
    dico["position"].append(i)
    dico["qualy"].append(df_lec["quali"].value_counts().get(i, 0))
    dico["result"].append(df_lec["result"].value_counts().get(i, 0))
df_lec = pd.DataFrame(dico)
dfq = df_lec[df_lec["qualy"] > 0]
dfr = df_lec[df_lec["result"] > 0]

req         WARNING 	DEFAULT CACHE ENABLED! (175.3 MB) C:\Users\Shayane\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Dutch Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '16', '81', '55', '11', '63', '44', '10', '14', '27', '3', '18', '23', '

15
15
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1]


## Last Race

### Race

In [34]:
# get the average lap time per driver
px.scatter(average_lap_time, x="Driver", y="LapTime(s)", title="Average lap time per driver", template="plotly_dark")

In [35]:
# line plot of the lap times per driver per lap with color per driver
px.line(driver_laps, x="LapNumber", y="LapTime(s)", color="Driver", title="Lap times per driver per lap", template="plotly_dark", color_discrete_map=driver_colors)

In [36]:
# position evolution per driver per lap (reverse y-axis)
fig = px.line(driver_laps, x="LapNumber", y="Position", color="Driver", title="Position evolution per driver", template="plotly_dark", color_discrete_map=driver_colors)
fig.update_yaxes(autorange="reversed")
fig

In [37]:
# type of tyre per driver per lap (horizontal bar plot)
fig = px.bar(stints, x="StintLength", y="Driver", color="Compound", title="Stint length per driver", template="plotly_dark", color_discrete_map=compound_colors, orientation="h")
fig.update_yaxes(tickmode="linear")
fig

In [9]:
# laptime depending on tire compound
px.scatter(driver_laps, x="LapNumber", y="LapTime(s)", color="Compound", title="Lap time per lap depending on tire compound", template="plotly_dark", color_discrete_map=compound_colors)

In [38]:
# gap to winner
fig = px.line(driver_laps, x="LapNumber", y="GapToWinner", color="Driver",
              title="Écart temporel au vainqueur par pilote",
              template="plotly_dark", color_discrete_map=driver_colors)
fig.update_yaxes(autorange="reversed")
fig

### Qualifying

In [39]:
# qualifying time
fig = px.scatter(fastest_laps, x="Driver", y="LapTime", title="Qualifying lap time delta to pole", template="plotly_dark", color="Driver", color_discrete_map=driver_colors)

# lines constants
hundredsevenrule = pole_lap["LapTime"] * 1.07
hundredsevenrule = hundredsevenrule.total_seconds()
q1toq2 = 15
q2toq3 = 10

# add lines
fig.add_hline(y=hundredsevenrule, line_dash="dot", line_color="red", annotation_text="107% rule", annotation_position="bottom right") # 107% rule
fig.add_vline(x=q1toq2, line_dash="dot", line_color="red", annotation_text="eliminated in Q1", annotation_position="bottom right") # Q1 to Q2
fig.add_vline(x=q2toq3, line_dash="dot", line_color="red", annotation_text="eliminated in Q2", annotation_position="bottom right") # Q2 to Q3

In [40]:
# 2 fastest drivers of qualy comparison
px.scatter(qualy2, x='X', y='Y', color='Driver', color_discrete_map=colormap, title=f"{r1["Abbreviation"][0]} vs {r2["Abbreviation"][0]} in qualy", template="plotly_dark")

C:\Users\Shayane\AppData\Local\Temp\ipykernel_13796\966582036.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## Global

### General

In [41]:
# driver standing
px.line(global_df, x="raceIndex", y="standing", color="driver", title="Driver standing", template="plotly_dark", markers=True, color_discrete_map=driver_colors)

In [43]:
# team standing
px.line(global_team_df, x="raceIndex", y="standing", color="team", title="Team standing", template="plotly_dark", markers=True, color_discrete_map=team_colors)

In [44]:
# qualy result through the season
fig = px.line(global_df, x="raceIndex", y="quali", color="driver", title="Quali result through the season", template="plotly_dark", markers=True, color_discrete_map=driver_colors)
fig.update_yaxes(autorange="reversed")

In [45]:
# race result through the season
fig = px.line(global_df, x="raceIndex", y="result", color="driver", title="Race result through the season", template="plotly_dark", markers=True, color_discrete_map=driver_colors)
fig.update_yaxes(autorange="reversed")

In [46]:
# global per driver
px.imshow(global_df.pivot(index="driver", columns="raceIndex", values="quali"), title="Global qualy per driver", template="plotly_dark", width=1200, height=800, color_continuous_scale="Viridis_r")


In [47]:
# global result per driver
px.imshow(global_df.pivot(index="driver", columns="raceIndex", values="result"), title="Global result per driver", template="plotly_dark", width=1200, height=800, color_continuous_scale="Viridis_r")

In [48]:
# pie chart top 1 qualifying
df_q1 = global_df[global_df["quali"] == 1]
px.pie(df_q1, names="driver", values="quali", title="Top 1 qualifying", template="plotly_dark", color="driver", color_discrete_map=driver_colors)

In [49]:
# pie chart p1
df_p1 = global_df[global_df["result"] == 1]
px.pie(df_p1, names="driver", values="result", title="Top Driver P1", template="plotly_dark", color="driver", color_discrete_map=driver_colors)

In [50]:
# pie chart podium
df_podium = global_df[global_df["result"] <= 3]
px.pie(df_podium, names="driver", values="result", title="Podium", template="plotly_dark", color="driver", color_discrete_map=driver_colors)

### Leclerc

In [53]:
# bar chart leclerc qualy result
px.bar(dfq, x="position", y="qualy", title="Leclerc qualy result", template="plotly_dark")

In [54]:
# bar chart leclerc race result
px.bar(dfr, x="position", y="result", title="Leclerc race result", template="plotly_dark")